In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0) 

color_ranges = {
    "Red1": (np.array([0, 120, 70], dtype=np.uint8), np.array([10, 255, 255], dtype=np.uint8)),
    "Red2": (np.array([170, 120, 70], dtype=np.uint8), np.array([180, 255, 255], dtype=np.uint8)),
    "Yellow": (np.array([20, 100, 100], dtype=np.uint8), np.array([30, 255, 255], dtype=np.uint8)),
    "Blue": (np.array([90, 100, 100], dtype=np.uint8), np.array([130, 255, 255], dtype=np.uint8)),
    "Green": (np.array([35, 100, 100], dtype=np.uint8), np.array([85, 255, 255], dtype=np.uint8))
}

kernel_sizes = {
    "Red": (3, 3),
    "Yellow": (2, 2),
    "Blue": (3, 3),
    "Green": (3, 3)
}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    masks = {color: cv2.inRange(hsv, lower, upper) for color, (lower, upper) in color_ranges.items()}
    masks["Red"] = masks["Red1"] + masks["Red2"] 
    del masks["Red1"], masks["Red2"] 
    
    contours_by_color = {}
    for color, mask in masks.items():
        kernel = np.ones(kernel_sizes.get(color, (3, 3)), np.uint8)
        
        # Determine erosion/dilation iterations dynamically
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        num_contours = len(contours)
        erode_iters = 1 if num_contours > 10 else 2
        dilate_iters = 1 if num_contours < 5 else 2

        mask = cv2.erode(mask, kernel, iterations=erode_iters)
        mask = cv2.dilate(mask, kernel, iterations=dilate_iters)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Apply morphological closing
        
        # Edge detection and contour finding
        edges = cv2.Canny(mask, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_by_color[color] = contours
    
    for color, contours in contours_by_color.items():
        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue
            
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            num_sides = len(approx)
            shape = "Unknown"

            if num_sides == 3:
                shape = "Triangle"
            elif num_sides == 4:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                shape = "Square" if 0.95 <= aspect_ratio <= 1.05 else "Rectangle"
            elif num_sides == 5:
                shape = "Pentagon"
            elif num_sides == 6:
                shape = "Hexagon"
            else:
                area = cv2.contourArea(contour)
                perimeter = cv2.arcLength(contour, True)
                if perimeter > 0:
                    circularity = 4 * np.pi * (area / (perimeter ** 2))
                    if 0.7 <= circularity <= 1.2:
                        shape = "Circle"
            
            x, y, w, h = cv2.boundingRect(contour)
            cv2.drawContours(frame, [contour], -1, (255, 255, 0), 2)
            text = f"{color} {shape}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    for color, mask in masks.items():
        cv2.imshow(f"{color} Mask", mask)
    cv2.imshow("2D Object Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()